In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import graph as gr
import utils as ut
import feature_selection as fs
import classification as cl
import os.path
import matplotlib.pyplot as plt
from pprint import pprint
import importlib
import community
import seaborn as sns
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import copy
import itertools
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Reshape
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using Theano backend.
C:\Users\faisal\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
importlib.reload(gr)

<module 'graph' from 'C:\\Users\\faisal\\PycharmProjects\\linkPrediction\\graph.py'>

In [175]:
domain = ['apnea','obesity']
filepath=['dataset/obesity-all,3.csv',
         'dataset/obesity-distinct_keyword.csv']
graphpath='C:/Users/faisal/PycharmProjects/linkPrediction/graphs'
datapath='C:/Users/faisal/PycharmProjects/linkPrediction/dataframes'
modelpath='C:/Users/faisal/PycharmProjects/linkPrediction/models'
column_split=['keyword','author_name','affiliation_1','affiliation_2','country']
#time=[parent_start_year, train_start_year, test_start_year, total_test_period_in_year, total_year_in_each_iteration]
time=[1991,1991,1996,1,1,20]
df, key_list = ut.load_dataset(filepath, column_split)

In [20]:
for tout in range(1991,2010,6):
    time =  [tout,tout+1,tout+6,1,1,20]
    gr.dynamic_train_test_graph_build(df, key_list, graphpath, time)

204
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992.gpickle saved, nodes: 64 edges: 56
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1993.gpickle saved, nodes: 63 edges: 62
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1994.gpickle saved, nodes: 84 edges: 82
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1995.gpickle saved, nodes: 108 edges: 137
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1996.gpickle saved, nodes: 127 edges: 172
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997.gpickle saved, nodes: 204 edges: 243
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\parent_graph_1991-1992.gpickle saved, nodes: 52 edges: 53
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992-1997.gpickle saved, nodes: 204 edges: 417
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997-1998.gpickle saved, nodes: 204 edges: 243
401
C:/Users/faisa

In [162]:
graphs = {}
for tout in range(1991,2010,6):
    time =  [tout,tout+1,tout+6,1,1,20]
    g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
    graphs[tout]=g_parent
    graphs.update(g_train)

C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992.gpickle loaded, nodes: 64 edges: 56
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1993.gpickle loaded, nodes: 63 edges: 62
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1994.gpickle loaded, nodes: 84 edges: 82
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1995.gpickle loaded, nodes: 108 edges: 137
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1996.gpickle loaded, nodes: 127 edges: 172
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997.gpickle loaded, nodes: 204 edges: 243
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\parent_graph_1991-1992.gpickle loaded, nodes: 52 edges: 53
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992-1997.gpickle loaded, nodes: 204 edges: 417
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997-1998.gpickle loaded, nodes: 204 edges: 243
C:/Users/fais

In [163]:
k = set()
ko=set()
e = set()
eo = set()
gr_ev = []
for year,graph in graphs.items():
    k = set(graph.nodes())
    k_n = k.difference(ko)
    k_o = k.difference(k_n)
    e = set(graph.edges())
    e_ooo = e.intersection(eo)
    e_nn_no = e.difference(eo)
    e_oon = set()
    e_nn = set()
    e_no = set()
    for es in e_nn_no:
        diff = len(set(es).difference(ko))
        if diff==0:
            e_oon.add(es)
        if diff==2:
            e_nn.add(es)
        if diff==1:
            e_no.add(es)
#     print('year:',year)
#     print('k:',len(k),'kn:',len(k_n),'ko:',len(k_o))
#     print('e:',len(e),'e_ooo:',len(e_ooo),'e_oon:',len(e_oon),'e_no:',len(e_no),'e_nn:',len(e_nn))
    gr_ev.append([year,len(k),len(k_n),len(k_o),len(e),len(e_ooo),len(e_oon),len(e_no),len(e_nn)])
    ko.update(k)
    eo.update(e)
cols = ['year', 'k', 'kn', 'ko', 'e', 'e_ooo', 'e_oon', 'e_no', 'e_nn',]
gr_ke = pd.DataFrame(gr_ev, columns=cols) 

In [164]:
importlib.reload(fs)
importlib.reload(cl)
type_ev = []
for tout in range(1991,2010,6):
    time =  [tout,tout+1,tout+6,1,1,20]
    g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
    freq=10
    #time=[2001,2007,2015,1,1,10]
    train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                             g_test,
                                                                                                             g_parent,
                                                                                                             g_train_static, 
                                                                                                             g_test_static,  
                                                                                                             time, 
                                                                                                             freq)
    node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)
    for year,node_f in node_feature.items():
        k_gp = node_f.loc[node_f['node_type_aut']==20,['node_type_aut']].count()
        k_p = node_f.loc[node_f['node_type_aut']==5,['node_type_aut']].count()
        k_c = node_f.loc[node_f['node_type_aut']==3,['node_type_aut']].count()
        k_g = node_f.loc[node_f['node_type_aut']==1,['node_type_aut']].count()
        #print('k_gp:',k_gp[0],'k_p:',k_p[0],'k_c:',k_c[0],'k_g:',k_g[0])
        e_gpgp1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==400),['type']].count()
        e_gpgp0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==400),['type']].count()
        e_gpp1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==100),['type']].count()
        e_gpp0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==100),['type']].count()
        e_gpc1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==60),['type']].count()
        e_gpc0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==60),['type']].count()
        e_gpg1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==20),['type']].count()
        e_gpg0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==20),['type']].count()
        total = train_data[year].loc[train_data[year]['label']==1,['type']].count()
        tot_gp = e_gpgp1[0] + e_gpp1[0] + e_gpc1[0]+ e_gpg1[0]
        rat = tot_gp*100/total[0]
        #print('gpgp:',e_gpgp1[0],'gpp:',e_gpp1[0],'gpc:',e_gpc1[0],'gpg',e_gpg1[0],'total:',total[0],'tot_p:',tot_gp,'rat:',rat)
        type_ev.append([year,k_gp[0],k_p[0],k_c[0],k_g[0],e_gpgp1[0],e_gpp1[0],e_gpc1[0],e_gpg1[0],tot_gp,total[0],rat])
cols = ['year','k_gp','k_p','k_c','k_g','e_gpgp1','e_gpp1','e_gpc1','e_gpg1','tot_gp','total','rat']
gr_type = pd.DataFrame(type_ev, columns=cols) 

C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992.gpickle loaded, nodes: 64 edges: 56
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1993.gpickle loaded, nodes: 63 edges: 62
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1994.gpickle loaded, nodes: 84 edges: 82
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1995.gpickle loaded, nodes: 108 edges: 137
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1996.gpickle loaded, nodes: 127 edges: 172
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997.gpickle loaded, nodes: 204 edges: 243
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\parent_graph_1991-1992.gpickle loaded, nodes: 52 edges: 53
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992-1997.gpickle loaded, nodes: 204 edges: 417
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997-1998.gpickle loaded, nodes: 204 edges: 243
pos in time s

In [165]:
importlib.reload(fs)
importlib.reload(cl)
miss_ev = []
for tout in range(1991,2010,6):
    time =  [tout,tout+1,tout+6,1,1,20]
    ts = time[1]
    te = time[2]
    g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
    freq=10
    #time=[2001,2007,2015,1,1,10]
    train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                             g_test,
                                                                                                             g_parent,
                                                                                                             g_train_static, 
                                                                                                             g_test_static,  
                                                                                                             time, 
                                                                                                             freq)
    node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)
    static_pos = set(train_data_static[(train_data_static['label']==1)]['row_name'])
    dynamic_pos = set()
    for t in range(ts,te,1):
        dynamic_pos.update(train_data[t][(train_data[t]['label']==1)]['row_name'])
    transitory_edges = static_pos.difference(dynamic_pos)
    transitory_nodes = set()
    for row in transitory_edges:
        transitory_nodes.add(row[0])
        transitory_nodes.add(row[1]) 
    nodes_te =set()
    nodes_te1 = set()
    nodes_teo = set()
    for node in transitory_nodes:
        if (te-1) in set(g_train_static.nodes[node]['year']):
            nodes_te.add(node)
        if (te-2) in set(g_train_static.nodes[node]['year']):
            nodes_te1.add(node)
    nodes_te1 = nodes_te1.difference(nodes_te)        
    nodes_teo = transitory_nodes.difference(nodes_te,nodes_te1)
    #print('n_te:',len(nodes_te),'n_te1:',len(nodes_te1),'n_teo:',len(nodes_teo),'n_tot:',len(transitory_nodes))
    e_tete1= set()
    e_teteo= set()
    for row in transitory_edges:
        diff = len(set(row).difference(nodes_te))
        if diff==1:
            diff1 = len(set(row).difference(nodes_te1))
            if diff1 ==1:
                e_tete1.add(row)
            if diff1 ==2:
                e_teteo.add(row)
    tot_te = len(e_tete1)+len(e_teteo)
    rat = tot_te*100/len(transitory_edges)
    #print('e_tete1:',len(e_tete1),'e_teteo:',len(e_teteo),'tot_e:',tot_te,'total:',len(transitory_edges),'rat:',rat) 
    miss_ev.append([ts,te-1,len(nodes_te),len(nodes_te1),len(nodes_teo),len(transitory_nodes),
                    len(e_tete1),len(e_teteo),tot_te,len(transitory_edges),rat])
cols = ['ts','te','n_te','n_te1','n_teo','n_trans','e_tete1','e_teteo','tot_te','tot_e_trans','rat']
gr_miss=pd.DataFrame(miss_ev, columns=cols)

C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992.gpickle loaded, nodes: 64 edges: 56
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1993.gpickle loaded, nodes: 63 edges: 62
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1994.gpickle loaded, nodes: 84 edges: 82
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1995.gpickle loaded, nodes: 108 edges: 137
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1996.gpickle loaded, nodes: 127 edges: 172
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997.gpickle loaded, nodes: 204 edges: 243
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\parent_graph_1991-1992.gpickle loaded, nodes: 52 edges: 53
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\train_graph_1992-1997.gpickle loaded, nodes: 204 edges: 417
C:/Users/faisal/PycharmProjects/linkPrediction/graphs\test_graph_1997-1998.gpickle loaded, nodes: 204 edges: 243
pos in time s

In [172]:
gr_type
print(gr_type.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} &  year &  k\_gp &  k\_p &  k\_c &  k\_g &  e\_gpgp1 &  e\_gpp1 &  e\_gpc1 &  e\_gpg1 &  tot\_gp &  total &   rat \\
\midrule
0  &  1992 &    12 &   20 &   26 &    6 &        1 &       8 &       4 &       2 &      15 &     23 & 65.22 \\
1  &  1993 &    13 &   23 &   25 &    2 &        4 &       3 &      10 &       1 &      18 &     25 & 72.00 \\
2  &  1994 &    13 &   28 &   38 &    5 &        3 &       2 &      17 &       2 &      24 &     49 & 48.98 \\
3  &  1995 &    16 &   39 &   42 &   11 &        3 &      14 &       9 &       4 &      30 &     65 & 46.15 \\
4  &  1996 &    19 &   59 &   37 &   12 &        3 &      17 &      12 &       7 &      39 &     83 & 46.99 \\
5  &  1998 &    16 &   54 &   67 &   14 &        6 &      21 &      16 &       5 &      48 &    106 & 45.28 \\
6  &  1999 &    17 &   47 &   76 &   17 &        3 &      14 &      21 &      13 &      51 &    122 & 41.80 \\
7  &  2000 &    19 &   66 &   69 &   32 &        5 &  

In [173]:
gr_miss
print(gr_miss.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{} &    ts &    te &  n\_te &  n\_te1 &  n\_teo &  n\_trans &  e\_tete1 &  e\_teteo &  tot\_te &  tot\_e\_trans &   rat \\
\midrule
0 &  1992 &  1996 &    26 &      9 &     18 &       53 &       11 &       22 &      33 &           39 & 84.62 \\
1 &  1998 &  2002 &    36 &     24 &     32 &       92 &       19 &       21 &      40 &           55 & 72.73 \\
2 &  2004 &  2008 &    69 &     43 &     51 &      163 &       38 &       44 &      82 &          104 & 78.85 \\
3 &  2010 &  2014 &    70 &     56 &     78 &      204 &       33 &       47 &      80 &          119 & 67.23 \\
\bottomrule
\end{tabular}



In [174]:
gr_ke
print(gr_ke.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  year &    k &   kn &   ko &     e &  e\_ooo &  e\_oon &  e\_no &  e\_nn \\
\midrule
0  &  1991 &   52 &   52 &    0 &    53 &      0 &      0 &     0 &    53 \\
1  &  1992 &   64 &   37 &   27 &    56 &      2 &     12 &    33 &     9 \\
2  &  1993 &   63 &   27 &   36 &    62 &      2 &     17 &    30 &    13 \\
3  &  1994 &   84 &   31 &   53 &    82 &      8 &     40 &    29 &     5 \\
4  &  1995 &  108 &   34 &   74 &   137 &     27 &     78 &    30 &     2 \\
5  &  1996 &  127 &   23 &  104 &   172 &     42 &     98 &    30 &     2 \\
6  &  1997 &  130 &   28 &  102 &   150 &     36 &     84 &    29 &     1 \\
7  &  1998 &  151 &   70 &   81 &   197 &     38 &     55 &    79 &    25 \\
8  &  1999 &  157 &   52 &  105 &   153 &     41 &     60 &    45 &     7 \\
9  &  2000 &  186 &   46 &  140 &   272 &     76 &    143 &    50 &     3 \\
10 &  2001 &  201 &   57 &  144 &   306 &     79 &    138 &    82 &     7 \\
11 &  2002 &  233 &   46 &

In [200]:
df.groupby('art_year').count()

,art_id,keyword,citation,title,author_name,affiliation_1,affiliation_2,country
art_year,,,,,,,,
2007,4849,4849,4849,4849,4849,4847,4719,4849
2008,5486,5486,5486,5486,5486,5484,5375,5486
2012,1491,1491,1491,1491,1491,1491,1486,1491
2013,1749,1749,1749,1749,1749,1746,1740,1745
2014,8115,8115,8115,8115,8115,8053,8061,8115
2015,8570,8570,8570,8570,8570,8492,8568,8566
2016,17,17,17,17,17,17,17,17


In [204]:
cols = ['year','total article']
obesity_stat=pd.DataFrame(d2, columns=cols)
print(obesity_stat.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrr}
\toprule
{} &  year &  total article \\
\midrule
0 &  2007 &           4849 \\
1 &  2008 &           5486 \\
2 &  2012 &           1491 \\
3 &  2013 &           1749 \\
4 &  2014 &           8115 \\
5 &  2015 &           8570 \\
6 &  2016 &             17 \\
\bottomrule
\end{tabular}

